In [150]:
import sys
sys.path.append("../")
from initial_dataSet import DataSet
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.cluster import KMeans

In [151]:
basedir = '../'
models=['CICDM','CICDM-A','CDGK','NCD','DINA']

model=models[4]
dataSet_list = ('ASSIST_0910', 'ASSIST_2017','JUNYI', 'MathEC')
save_list = ('a0910/', 'a2017/', 'junyi/', 'math_ec/')

dataSet_idx=3
data_set_name = dataSet_list[dataSet_idx]
dataSet = DataSet(basedir, data_set_name,build=True)

if model=='CICDM':
    read_dir=dataSet.save_result_dir
elif model=='CICDM-A':
    read_dir='./output/'+save_list[dataSet_idx]
elif model=='CDGK':
    read_dir='E:/PY_Project/知识点交互CDM/temp_Baseline/CDGK/output/'+save_list[dataSet_idx]
elif model=='NCD':
    read_dir='E:/PY_Project/知识点交互CDM/Baseline/NCD/output/'+save_list[dataSet_idx]
elif model=='DINA':
    read_dir='E:/PY_Project/知识点交互CDM/Baseline/DINA/output/'+save_list[dataSet_idx]
else:
    assert False,'模型名称错误'

cogn_state=np.loadtxt(read_dir+'cognitive_state.csv',delimiter=',')
record=dataSet.record.reset_index()

DataSet: MathEC


In [152]:
cluster_k=4

cluster=KMeans(n_clusters=cluster_k)
cluster.fit(cogn_state)
labels=cluster.labels_

In [153]:
record['cluster']=labels[record['user_id']-1]

In [154]:
item_mean_score=np.ones((cluster_k,dataSet.exercise_num))*-1e10
for cluster_num in range(cluster_k):
    clust_record=record.set_index('cluster').loc[cluster_num,:]
    mean_score=clust_record.groupby('item_id').mean()['score']
    item_mean_score[cluster_num,mean_score.index-1]=mean_score.values
item_mean_score[item_mean_score<-1e5]=np.nan

In [155]:
centers=cluster.cluster_centers_
cent_abs_list=[]
for i in range(cluster_k):
    for j in range(i+1,cluster_k):
        diff=centers[i]-centers[j]
        diff=diff[~np.isnan(diff)]
        cent_abs_list.append(np.abs(diff).mean())

print(model,data_set_name)
print('熟练度区分度:',np.mean(cent_abs_list)/record['score'].mean())

DINA MathEC
熟练度区分度: 0.09348604608570495


In [156]:
abs_list=[]
for i in range(cluster_k):
    for j in range(i+1,cluster_k):
        diff=item_mean_score[i]-item_mean_score[j]
        diff=diff[~np.isnan(diff)]
        abs_list.append(np.abs(diff).mean())

print(model,data_set_name)
print('得分区分度:',np.mean(abs_list)/record['score'].mean())

DINA MathEC
得分区分度: 0.35268321720054474
